**IMPORTS**

In [1]:
import cobra
from cobra.io import (
    load_json_model,
    save_json_model,
    load_matlab_model,
    save_matlab_model,
    read_sbml_model,
    write_sbml_model,
)
from cobra.flux_analysis import flux_variability_analysis
import re

In [16]:
M_xanthus = read_sbml_model("../M_xanthus_model_V2.sbml")
M_xanthus

Name,myxo_model
Memory address,71f544b04c20
Number of metabolites,1280
Number of reactions,1367
Number of genes,1192
Number of groups,0
Objective expression,1.0*OF_BIOMASS - 1.0*OF_BIOMASS_reverse_80d2e
Compartments,"c, e"


In [3]:
E_coli = read_sbml_model("../E_coli_model.sbml")
E_coli

Name,iML1515
Memory address,783fd92e62d0
Number of metabolites,1877
Number of reactions,2714
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


**TESTING**

In [15]:
liste = []
for i in M_xanthus.reactions:
    if 'and' in i.gene_reaction_rule:
        liste.append(i.gene_reaction_rule)
    elif 'or' in i.gene_reaction_rule and 'and' not in i.gene_reaction_rule:
       result = re.split(r'or', i.gene_reaction_rule)
       for j in result:
            liste.append(j)
    elif i.gene_reaction_rule == "":
        continue

print(liste)

['MXAN_4353 ', ' MXAN_5925', 'MXAN_4564 and MXAN_4565', 'MXAN_3261 and MXAN_7449 and MXAN_1572', 'MXAN_3644 ', ' MXAN_3644', 'MXAN_6374 and MXAN_3518 and MXAN_2536 and MXAN_1573 and MXAN_0225 and MXAN_7148 and MXAN_1528 and MXAN_0216 and MXAN_2959', 'MXAN_2666 and MXAN_2667', 'MXAN_3800 and MXAN_2179', 'MXAN_5327 and MXAN_5328', '(MXAN_5371 and MXAN_5136 and MXAN_3800 and MXAN_6987) or MXAN_5136', 'MXAN_1437 and MXAN_2907', '(MXAN_6714 and MXAN_7084) or MXAN_4963 or MXAN_6822 or MXAN_1160 or MXAN_1160', 'MXAN_2336 and MXAN_2337', 'MXAN_3799 and MXAN_3757', 'MXAN_2079 ', ' MXAN_2079', 'MXAN_3222 and MXAN_4010 and MXAN_4576 and MXAN_5465 and MXAN_1686 and MXAN_1062 and MXAN_5851 and MXAN_1633 and MXAN_2367 and MXAN_6984 and MXAN_3661 and MXAN_3054 and MXAN_4161 and MXAN_6704 and MXAN_6182 and MXAN_2615 and MXAN_3980 and MXAN_1523 and MXAN_2839 and MXAN_5385 and MXAN_0262 and MXAN_5917 and MXAN_4594 and MXAN_1390 and MXAN_1237 and MXAN_1079 and MXAN_4566 and MXAN_6424 and MXAN_0517 and MX

In [45]:
M_xanthus.metabolites.pi_e
for i in M_xanthus.exchanges._dict:
    print(i + ": " + M_xanthus.reactions.get_by_id(i).name)

EX_malt_e: Exchange for Maltose [e]
EX_his_L_e: Exchange for L-Histidine [e]
EX_cd2_e: Exchange for Cd2+ [e]
EX_ferrich_e: Exchange for Ferrichrome [e]
EX_tttnt_e: Exchange for Tetrathionate [e]
EX_metox_e: Exchange for L-Methionine S-oxide [e]
EX_btn_e: Exchange for BIOT [e]
EX_met_L_e: Exchange for L-Methionine [e]
EX_mg2_e: Exchange for Mg [e]
EX_ac_e: Exchange for Acetate [e]
EX_arsenb_e: Exchange for ARSENOBETAINE [e]
EX_cbl1_e: Exchange for Cbl [e]
EX_cgly_e: Exchange for Cys-Gly [e]
EX_fum_e: Exchange for Fumarate [e]
EX_h2s_e: Exchange for H2S [e]
EX_so4_e: Exchange for Sulfate [e]
EX_pydx_e: Exchange for Pyridoxal [e]
EX_spmd_e: Exchange for Spermidine [e]
EX_ppi_e: Exchange for PPi [e]
EX_mnl_e: Exchange for D-Mannitol [e]
EX_Pb_e: Exchange for Pb [e]
EX_pro_L_e: Exchange for L-Proline [e]
EX_glyc3p_e: Exchange for Glycerol-3-phosphate [e]
EX_orn_e: Exchange for Ornithine [e]
EX_salcn_e: Exchange for Salicin [e]
EX_pi_e: Exchange for Phosphate [e]
EX_acgam_e: Exchange for N-A

In [135]:
M_xanthus.reactions.OF_BIOMASS.reaction

'0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c 

In [67]:
E_coli.reactions.BIOMASS_Ec_iML1515_WT_75p37M.reaction

'0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 70.028756 h2o_c + 0.000223 hemeO_c + 0.092056 his__L_c + 0.282306 ile__L_c + 0.18569 k_c + 0.437778 leu__L_c + 3e-06 lipopb_c + 0.333448 lys__L_c + 3.1e-05 malcoa_c + 0.149336 met__L_c + 0.008253 mg2_c + 0.000223 mlthf_c + 0.000658 mn2_c + 7e-06 mobd_c + 

In [10]:
# enter_e = all imported are exported
M_xanthus.metabolites.enter_e.summary()
M_xanthus.metabolites.feenter_e
M_xanthus.reactions.rxn08502_e

Reaction identifier,rxn08502_e
Name,enterobactin Fe(III) binding (spontaneous) [e]
Memory address,0x71f544007cb0
Stoichiometry,Fe3_e + enter_e <=> feenter_e fe3 [e] + Enterobactin [e] <=> Fe-enterochlin [e]
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [ ]:
# mobd = dead end ?
M_xanthus.metabolites.Adenylated_molybdopterin_c.summary()
M_xanthus.metabolites.Adenylated_molybdopterin_c
M_xanthus.reactions.rxn16583_c

Reaction identifier,rxn16583_c
Name,adenylyl-molybdopterin:molybdate molybdate transferase (AMP-forming) [c]
Memory address,0x783fd8cfea20
Stoichiometry,Adenylated_molybdopterin_c + mobd_c --> Molybdoenzyme_molybdenum_cofactor_c + amp_c + h2o_c + h_c Adenylated molybdopterin [c] + Molybdate [c] --> Molybdoenzyme molybdenum cofactor [c] + AMP [c] + H2O [c] + H+ [c]
GPR,MXAN_4212 and MXAN_7413
Lower bound,0.0
Upper bound,1000.0


In [ ]:
# btn_e = dead end ?
M_xanthus.metabolites.btn_cbxl_c.summary()  # btn_co2_c - btn_cbxl_c
M_xanthus.metabolites.btn_cbxl_c
M_xanthus.reactions.rxn06864_c  # rxn06672_c - rxn06673_c

Reaction identifier,rxn06864_c
Name,Biotinyl-5'-AMP:apo-[carboxylase] ligase (AMP-forming) [c]
Memory address,0x783fd93f1430
Stoichiometry,Apo_carboxylase_c + btamp_c <=> amp_c + btn_cbxl_c + h_c Apo-[carboxylase] [c] + Biotinyl-5-AMP [c] <=> AMP [c] + Holo-[carboxylase] [c] + H+ [c]
GPR,MXAN_4152
Lower bound,-1000.0
Upper bound,1000.0


**SUMMARY**

In [4]:
M_xanthus.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Fe3_e,EX_Fe3_e,0.3365,0,0.00%
acac_e,EX_acac_e,348.5,4,8.47%
acald_e,EX_acald_e,1000,2,12.16%
ca2_e,EX_ca2_e,0.1122,0,0.00%
cgly_e,EX_cgly_e,0.1122,5,0.00%
cl_e,EX_cl_e,0.1122,0,0.00%
cobalt2_e,EX_cobalt2_e,0.1122,0,0.00%
cu2_e,EX_cu2_e,0.1122,0,0.00%
etoh_e,EX_etoh_e,14.28,2,0.17%
fru_e,EX_fru_e,12.4,6,0.45%


In [13]:
E_coli.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.06116,0,0.00%
cl_e,EX_cl_e,0.06116,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0002938,0,0.00%
cu2_e,EX_cu2_e,0.008332,0,0.00%
fe2_e,EX_fe2_e,0.1887,0,0.00%
glc__D_e,EX_glc__D_e,105.7,6,100.00%
k_e,EX_k_e,2.294,0,0.00%
mg2_e,EX_mg2_e,0.1019,0,0.00%
mn2_e,EX_mn2_e,0.00812,0,0.00%
mobd_e,EX_mobd_e,8.226E-05,0,0.00%


**METHOD 2**

Add same reaction but reverse to both model, representing the "stealing" of the metabolites. The flux of E_coli should be the reverse of the flux of M_xanthus

- Add exchange reaction of importing mtabolite used for BIOMASS prod: PR_BIOMASS (in M.xanthus)
- Add exchange reaction of exporting BIOMASS: PR_BIOMASS (in E.coli)
- Add constrains to one model to force it to have a same flux (but reversed) as other model

For Escherichia coli

In [27]:
E_coli.reactions.BIOMASS_Ec_iML1515_WT_75p37M.reaction

'0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 70.028756 h2o_c + 0.000223 hemeO_c + 0.092056 his__L_c + 0.282306 ile__L_c + 0.18569 k_c + 0.437778 leu__L_c + 3e-06 lipopb_c + 0.333448 lys__L_c + 3.1e-05 malcoa_c + 0.149336 met__L_c + 0.008253 mg2_c + 0.000223 mlthf_c + 0.000658 mn2_c + 7e-06 mobd_c + 

In [131]:
Ereaction = cobra.Reaction("PR_BIOMASS")
Ereaction.name = "Predation Biomass"
Ereaction.subsystem = ""
Ereaction.lower_bound = 0.0  # This is the default
Ereaction.upper_bound = 1000.0  # This is the default

In [ ]:
spmd_c = (
    E_coli.metabolites.spmd_c
)  # carbon source from exchange used in biomass reaction
pheme_c = E_coli.metabolites.pheme_c
val_L_c = E_coli.metabolites.val__L_c  # imported AA
ile_L_c = E_coli.metabolites.ile__L_c
leu_L_c = E_coli.metabolites.leu__L_c
lys_L_c = E_coli.metabolites.lys__L_c
his_L_c = E_coli.metabolites.his__L_c
cys_L_c = E_coli.metabolites.cys__L_c
gly_c = E_coli.metabolites.gly_c
cgly_c = E_coli.metabolites.cgly_c
pro_L_c = E_coli.metabolites.pro__L_c
alaala_c = E_coli.metabolites.alaala_c
glu_L_c = E_coli.metabolites.glu__L_c
met_L_c = E_coli.metabolites.met__L_c
thr_L_c = E_coli.metabolites.thr__L_c
fru_c = E_coli.metabolites.fru_c  # import fructose
man_c = E_coli.metabolites.man_c
# malt_c = E_coli.metabolites.malt_c
acald_c = E_coli.metabolites.acald_c  # import FA
acac_c = E_coli.metabolites.acac_c
# glyc3p_c = E_coli.metabolites.glyc3p_c
# gam_c = E_coli.metabolites.gam_c
fum_c = E_coli.metabolites.fum_c  # should put directly the amino acids? asp and arg?
orn_c = E_coli.metabolites.orn_c
# succ_c = E_coli.metabolites.succ_c
pi_c = E_coli.metabolites.pi_c  # from research
zn2_c = E_coli.metabolites.zn2_c  # Metals
mn2_c = E_coli.metabolites.mn2_c
mg2_c = E_coli.metabolites.mg2_c
k_c = E_coli.metabolites.k_c
cu2_c = E_coli.metabolites.cu2_c
cobalt2_c = E_coli.metabolites.cobalt2_c
cl_c = E_coli.metabolites.cl_c
ca2_c = E_coli.metabolites.ca2_c
so4_c = E_coli.metabolites.so4_c
fe3_c = E_coli.metabolites.fe3_c

Ereaction.add_metabolites(
    {
        spmd_c: -1,
        pheme_c: -1,
        val_L_c: -1,
        ile_L_c: -1,
        leu_L_c: -1,
        lys_L_c: -1,
        his_L_c: -1,
        cys_L_c: -1,
        gly_c: -1,
        cgly_c: -1,
        pro_L_c: -1,
        alaala_c: -1,
        fru_c: -1,
        man_c: -1,
        # Smalt_c: -1,
        glu_L_c: -1,
        met_L_c: -1,
        thr_L_c: -1,
        # gam_c: -1,
        fum_c: -1,
        orn_c: -1,
        # succ_c: -1,
        acald_c: -1,
        acac_c: -1,
        # glyc3p_c: -1,
        pi_c: -1,
        zn2_c: -1,
        mn2_c: -1,
        mg2_c: -1,
        k_c: -1,
        cu2_c: -1,
        cobalt2_c: -1,
        cl_c: -1,
        ca2_c: -1,
        so4_c: -1,
        fe3_c: -1,
    }
)  # add the metabolites to the reaction

In [133]:
Ereaction.reaction

'acac_c + acald_c + alaala_c + ca2_c + cgly_c + cl_c + cobalt2_c + cu2_c + cys__L_c + fe3_c + fru_c + fum_c + glu__L_c + gly_c + his__L_c + ile__L_c + k_c + leu__L_c + lys__L_c + man_c + met__L_c + mg2_c + mn2_c + orn_c + pheme_c + pi_c + pro__L_c + so4_c + spmd_c + thr__L_c + val__L_c + zn2_c --> '

In [25]:
Ereaction.gene_reaction_rule = ""
Ereaction.genes

frozenset()

In [ ]:
# E_coli.add_reactions([Ereaction])

for Myxococcus xanthus

In [50]:
M_xanthus.reactions.OF_BIOMASS.reaction

'0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c 

In [119]:
Mreaction = cobra.Reaction("PR_BIOMASS")
Mreaction.name = "Predation Biomass"
Mreaction.subsystem = ""
Mreaction.lower_bound = 0.0  # This is the default
Mreaction.upper_bound = 1000.0  # This is the default

In [ ]:
spmd_e = (
    M_xanthus.metabolites.spmd_e
)  # carbon source from exchange used in biomass reaction
pheme_e = M_xanthus.metabolites.pheme_e
val_L_e = M_xanthus.metabolites.val_L_e  # imported AA
ile_L_e = M_xanthus.metabolites.ile_L_e
leu_L_e = M_xanthus.metabolites.leu_L_e
lys_L_e = M_xanthus.metabolites.lys_L_e
his_L_e = M_xanthus.metabolites.his_L_e
gly_cys_L_e = M_xanthus.metabolites.gly_cys_L_e
cgly_e = M_xanthus.metabolites.cgly_e
pro_L_e = M_xanthus.metabolites.pro_L_e
alaala_e = M_xanthus.metabolites.alaala_e
glu_L_c = M_xanthus.metabolites.glu_L_c
met_L_e = M_xanthus.metabolites.met_L_e
thr_L_e = M_xanthus.metabolites.thr_L_e
fru_e = M_xanthus.metabolites.fru_e  # import fructose
man_e = M_xanthus.metabolites.man_e
# malt_e = M_xanthus.metabolites.malt_e
acald_e = M_xanthus.metabolites.acald_e  # import FA
acac_e = M_xanthus.metabolites.acac_e
# glyc3p_e = M_xanthus.metabolites.glyc3p_e
# gam_e = M_xanthus.metabolites.gam_e
fum_e = M_xanthus.metabolites.fum_e  # should put directly the amino acids? asp and arg?
orn_e = M_xanthus.metabolites.orn_e
# succ_e = M_xanthus.metabolites.succ_e
pi_e = M_xanthus.metabolites.pi_e  # from research
zn2_e = M_xanthus.metabolites.zn2_e  # Metals
mn2_e = M_xanthus.metabolites.mn2_e
mg2_e = M_xanthus.metabolites.mg2_e
k_e = M_xanthus.metabolites.k_e
cu2_e = M_xanthus.metabolites.cu2_e
cobalt2_e = M_xanthus.metabolites.cobalt2_e
cl_e = M_xanthus.metabolites.cl_e
ca2_e = M_xanthus.metabolites.ca2_e
so4_e = M_xanthus.metabolites.so4_e
fe3_e = M_xanthus.metabolites.Fe3_e

Mreaction.add_metabolites(
    {
        spmd_e: 1,
        pheme_e: 1,
        val_L_e: 1,
        ile_L_e: 1,
        leu_L_e: 1,
        lys_L_e: 1,
        his_L_e: 1,
        gly_cys_L_e: 1,
        cgly_e: 1,
        pro_L_e: 1,
        alaala_e: 1,
        fru_e: 1,
        man_e: 1,
        # Smalt_e: 1,
        glu_L_c: 1,
        met_L_e: 1,
        thr_L_e: 1,
        # gam_e: 1,
        fum_e: 1,
        orn_e: 1,
        # succ_e: 1,
        acald_e: 1,
        acac_e: 1,
        # glyc3p_e: 1,
        pi_e: 1,
        zn2_e: 1,
        mn2_e: 1,
        mg2_e: 1,
        k_e: 1,
        cu2_e: 1,
        cobalt2_e: 1,
        cl_e: 1,
        ca2_e: 1,
        so4_e: 1,
        fe3_e: 1,
    }
)  # add the metabolites to the reaction

In [121]:
Mreaction.reaction

' --> Fe3_e + acac_e + acald_e + alaala_e + ca2_e + cgly_e + cl_e + cobalt2_e + cu2_e + fru_e + fum_e + glu_L_c + gly_cys_L_e + his_L_e + ile_L_e + k_e + leu_L_e + lys_L_e + man_e + met_L_e + mg2_e + mn2_e + orn_e + pheme_e + pi_e + pro_L_e + so4_e + spmd_e + thr_L_e + val_L_e + zn2_e'

In [47]:
Mreaction.gene_reaction_rule = ""
Mreaction.genes

frozenset()

In [ ]:
# M_xanthus.add_reactions([Mreaction])

Define the bound to 0 for metabolites in the reactions and not in the environment

In [ ]:
M_xanthus.reactions.EX_spmd_e.lower_bound = (
    0  # ? - not imported by E.coli / used for Biomass
)
M_xanthus.reactions.EX_pheme_e.lower_bound = (
    0  # amino acid - from Data / not imported by E.coli
)
M_xanthus.reactions.EX_val_L_e.lower_bound = 0  # amino acid - from Data
M_xanthus.reactions.EX_ile_L_e.lower_bound = 0  # amino acid - from Data
M_xanthus.reactions.EX_leu_L_e.lower_bound = 0  # amino acid - from Data
M_xanthus.reactions.EX_lys_L_e.lower_bound = 0  # amino acid
M_xanthus.reactions.EX_gly_cys_L_e.lower_bound = 0  # amino acid - from Data
M_xanthus.reactions.EX_cgly_e.lower_bound = 0  # amino acid - from Data
M_xanthus.reactions.EX_pro_L_e.lower_bound = 0  # amino acid - from Data
M_xanthus.reactions.EX_alaala_e.lower_bound = 0  # amino acid
M_xanthus.reactions.EX_glu_L_e.lower_bound = 0  # amino acid
M_xanthus.reactions.EX_met_L_e.lower_bound = 0  # amino acid - from Data
M_xanthus.reactions.EX_thr_L_e.lower_bound = 0  # amino acid
M_xanthus.reactions.EX_fru_e.lower_bound = 0  # sugar - from data / article
M_xanthus.reactions.EX_man_e.lower_bound = 0  # sugar - from data / article
# M_xanthus.reactions.EX_malt_e.lower_bound = 0
M_xanthus.reactions.EX_acald_e.lower_bound = 0  # FA - from article
M_xanthus.reactions.EX_acac_e.lower_bound = 0  # FA - from article
# M_xanthus.reactions.EX_glyc3p_e.lower_bound = 0
# M_xanthus.reactions.EX_gam_e.lower_bound = 0
M_xanthus.reactions.EX_fum_e.lower_bound = 0
M_xanthus.reactions.EX_orn_e.lower_bound = 0
# M_xanthus.reactions.EX_succ_e.lower_bound = 0
# M_xanthus.reactions.EX_pi_e.lower_bound = 0 # phosphate - from Data / in env
# M_xanthus.reactions.EX_zn2_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_mn2_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_mg2_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_k_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_cu2_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_cobalt2_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_cl_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_ca2_e.lower_bound = 0 # Metals - from Data / in env
# M_xanthus.reactions.EX_so4_e.lower_bound = 0 # sulfate - not imported by E.coli / in env
# M_xanthus.reactions.EX_Fe3_e.lower_bound = 0 # Metals - from data ? in env

Add constraints to for those to have the same flux

Can be done in dynamic env:
    
    FBA = M_xanthus.optimize()
    same_flux = E_coli.problem.Constraint(
        FBA['PR_BIOMASS'] - E_coli.reactions.PR_BIOMASS.flux_expression,
        lb=0,
        ub=0)
    E_coli.add_cons_vars(same_flux)

    FBA2 = E_coli.optimize()

In [ ]:
# same_flux = E_coli.problem.Constraint(
#    M_xanthus.reactions.PR_BIOMASS.flux_expression - E_coli.reactions.PR_BIOMASS.flux_expression,
#    lb=0,
#    ub=0)
# E_coli.add_cons_vars(same_flux)

**METHOD 4**

In [42]:
Biomass_e = cobra.Metabolite(
    "Biomass_e", formula="BIOMASS", name="Biomass [e]", compartment="e"
)

M_xanthus.add_metabolites(Biomass_e)  # add M_xanthus.metabolites.Biomass_e

Add both reactions: Biomass_e --> metabolites and EX_Biomass_e

In [43]:
Mreaction = cobra.Reaction("PR_BIOMASS")
Mreaction.name = "Predation Biomass"
Mreaction.subsystem = ""
Mreaction.lower_bound = 0.0  # This is the default
Mreaction.upper_bound = 1000.0  # This is the default

In [126]:
M_xanthus.reactions.OF_BIOMASS.reaction

'0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c 

In [44]:
Mreaction = cobra.Reaction("PR_BIOMASS")
Mreaction.name = "Predation Biomass"
Mreaction.subsystem = ""
Mreaction.lower_bound = 0.0  # This is the default
Mreaction.upper_bound = 1000.0  # This is the default

Biomass_e = M_xanthus.metabolites.Biomass_e

spmd_e = (
    M_xanthus.metabolites.spmd_e
)  # carbon source from exchange used in biomass reaction
pheme_e = M_xanthus.metabolites.pheme_e
val_L_e = M_xanthus.metabolites.val_L_e  # imported AA
ile_L_e = M_xanthus.metabolites.ile_L_e
leu_L_e = M_xanthus.metabolites.leu_L_e
lys_L_e = M_xanthus.metabolites.lys_L_e
his_L_e = M_xanthus.metabolites.his_L_e
gly_cys_L_e = M_xanthus.metabolites.gly_cys_L_e
cgly_e = M_xanthus.metabolites.cgly_e
pro_L_e = M_xanthus.metabolites.pro_L_e
alaala_e = M_xanthus.metabolites.alaala_e
glu_L_c = M_xanthus.metabolites.glu_L_c
met_L_e = M_xanthus.metabolites.met_L_e
thr_L_e = M_xanthus.metabolites.thr_L_e
fru_e = M_xanthus.metabolites.fru_e  # import fructose
malt_e = M_xanthus.metabolites.malt_e
gam_e = M_xanthus.metabolites.gam_e  # from here test
fum_e = M_xanthus.metabolites.fum_e
orn_e = M_xanthus.metabolites.orn_e
succ_e = M_xanthus.metabolites.succ_e
acald_e = M_xanthus.metabolites.acald_e
acac_e = M_xanthus.metabolites.acac_e
glyc3p_e = M_xanthus.metabolites.glyc3p_e

Mreaction.add_metabolites(
    {
        Biomass_e: -1,
        spmd_e: 1,
        pheme_e: 1,
        val_L_e: 1,
        ile_L_e: 1,
        leu_L_e: 1,
        lys_L_e: 1,
        his_L_e: 1,
        gly_cys_L_e: 1,
        cgly_e: 1,
        pro_L_e: 1,
        alaala_e: 1,
        fru_e: 1,
        malt_e: 1,
        glu_L_c: 1,
        met_L_e: 1,
        thr_L_e: 1,
        gam_e: 1,
        fum_e: 1,
        orn_e: 1,
        succ_e: 1,
        acald_e: 1,
        acac_e: 1,
        glyc3p_e: 1,
    }
)  # add the metabolites to the reaction

Mreaction.gene_reaction_rule = ""  # define which gene allow the reaction

M_xanthus.add_reactions([Mreaction])  # add the PR_Biomass reaction

M_xanthus.reactions.PR_BIOMASS

Ignoring reaction 'PR_BIOMASS' since it already exists.


Reaction identifier,PR_BIOMASS
Name,Predation Biomass
Memory address,0x754e7c15f120
Stoichiometry,Biomass_e --> acac_e + acald_e + alaala_e + cgly_e + fru_e + fum_e + gam_e + glu_L_c + gly_cys_L_e + glyc3p_e + his_L_e + ile_L_e + leu_L_e + lys_L_e + malt_e + met_L_e + orn_e + pheme_e + pro_L_e... Biomass [e] --> Acetoacetate [e] + Acetaldehyde [e] + Ala-Ala [e] + Cys-Gly [e] + D-Fructose [e] + Fumarate [e] + GLUM [e] + L-Glutamate [c] + Gly-Cys [e] + Glycerol-3-phosphate [e] + L-Histidine...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [45]:
Mreaction.reaction

'Biomass_e --> acac_e + acald_e + alaala_e + cgly_e + fru_e + fum_e + gam_e + glu_L_c + gly_cys_L_e + glyc3p_e + his_L_e + ile_L_e + leu_L_e + lys_L_e + malt_e + met_L_e + orn_e + pheme_e + pro_L_e + spmd_e + succ_e + thr_L_e + val_L_e'

In [46]:
Mreaction.gene_reaction_rule = ""
Mreaction.genes

frozenset()

In [47]:
M_xanthus.add_reactions([Mreaction])  # add the PR_Biomass reaction
M_xanthus.add_boundary(
    M_xanthus.metabolites.get_by_id("Biomass_e"), type="exchange"
)  # add the exchange reaction for Biomass_e

Ignoring reaction 'PR_BIOMASS' since it already exists.


Reaction identifier,EX_Biomass_e
Name,Biomass [e] exchange
Memory address,0x754deb4e5860
Stoichiometry,Biomass_e <=> Biomass [e] <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [48]:
M_xanthus.reactions.EX_spmd_e.lower_bound = (
    0  # define the bound to 0 for metabolites in the reactions
)
M_xanthus.reactions.EX_pheme_e.lower_bound = 0
M_xanthus.reactions.EX_val_L_e.lower_bound = 0
M_xanthus.reactions.EX_ile_L_e.lower_bound = 0
M_xanthus.reactions.EX_leu_L_e.lower_bound = 0
M_xanthus.reactions.EX_lys_L_e.lower_bound = 0
M_xanthus.reactions.EX_gly_cys_L_e.lower_bound = 0
M_xanthus.reactions.EX_cgly_e.lower_bound = 0
M_xanthus.reactions.EX_pro_L_e.lower_bound = 0
M_xanthus.reactions.EX_alaala_e.lower_bound = 0
M_xanthus.reactions.EX_glu_L_e.lower_bound = 0
M_xanthus.reactions.EX_met_L_e.lower_bound = 0
M_xanthus.reactions.EX_thr_L_e.lower_bound = 0
M_xanthus.reactions.EX_fru_e.lower_bound = 0
M_xanthus.reactions.EX_gam_e.lower_bound = 0  # test
M_xanthus.reactions.EX_fum_e.lower_bound = 0
M_xanthus.reactions.EX_orn_e.lower_bound = 0
M_xanthus.reactions.EX_malt_e.lower_bound = 0
M_xanthus.reactions.EX_succ_e.lower_bound = 0
M_xanthus.reactions.EX_acald_e.lower_bound = 0
M_xanthus.reactions.EX_acac_e.lower_bound = 0
M_xanthus.reactions.EX_glyc3p_e.lower_bound = 0

In [49]:
M_xanthus.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Biomass_e,EX_Biomass_e,988.1,0,0.00%
Fe3_e,EX_Fe3_e,0.325,0,0.00%
ca2_e,EX_ca2_e,0.1083,0,0.00%
cl_e,EX_cl_e,0.1083,0,0.00%
cobalt2_e,EX_cobalt2_e,0.1083,0,0.00%
cu2_e,EX_cu2_e,0.1083,0,0.00%
k_e,EX_k_e,0.1083,0,0.00%
mg2_e,EX_mg2_e,0.1083,0,0.00%
mn2_e,EX_mn2_e,0.1083,0,0.00%
nh4_e,EX_nh4_e,164.9,0,0.00%


In [50]:
M_xanthus.reactions.EX_metox_e

Reaction identifier,EX_metox_e
Name,Exchange for L-Methionine S-oxide [e]
Memory address,0x754df518de90
Stoichiometry,metox_e <=> L-Methionine S-oxide [e] <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [12]:
M_xanthus.metabolites.arg_L_c
M_xanthus.metabolites.asn_L_c
M_xanthus.metabolites.asp_L_c
M_xanthus.metabolites.gln_L_c
M_xanthus.metabolites.phe_L_c
M_xanthus.metabolites.ser_L_c
M_xanthus.metabolites.trp_L_c
M_xanthus.metabolites.tyr_L_c

Metabolite identifier,tyr_L_c
Name,L-Tyrosine [c]
Memory address,0x73a78bcfe670
Formula,C9H11NO3
Compartment,c
In 6 reaction(s),"rxn00527_c, rxn01315_c, OF_PROTEIN, rxn12847_c, rxn09310_c, rxn04989_c"
